In [39]:
import time
import math 
import requests
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as soup

In [40]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re

In [ ]:
browser = Browser('edge')

In [ ]:
url = "https://www.canadiantire.ca/en/promotions/clearance.html?page=1"
browser.visit(url)

In [ ]:
html = browser.html
html_soup = soup(html, 'html.parser')

In [ ]:
product_info = html_soup.find_all(class_='nl-product-card__primary-details')
product_data = []

for product in product_info:
    product_name_element = product.find('div', class_='nl-product-card__title')
    original_price_element = product.find('s', attrs={'aria-hidden': 'true'})
    clearance_price_element = product.find('span', class_='nl-price--total--red')
    rating_element = product.find('div', class_='bv_text')
    image_element = product.find('div', class_='nl-product-card__image-wrap')

    product_name = product_name_element.text.strip() if product_name_element else 'N/A'
    original_price = original_price_element.text.strip() if original_price_element else 'N/A'
    clearance_price = clearance_price_element.text.strip() if clearance_price_element else 'N/A'
    rating = rating_element.text.strip() if rating_element else 'N/A'
    product_code_element = product.find('p', class_='nl-product__code', attrs={'aria-hidden': 'true'})
    product_code = product_code_element.get_text(strip=True).lstrip('#') if product_code_element else 'N/A'

    product_data.append({
        'Product Name': product_name,
        'Original Price': original_price,
        'Clearance Price': clearance_price,
        'Rating': rating,
        'Product Code': product_code
    })

print(product_data)

In [ ]:
df1 = pd.DataFrame(product_data)

df1.head()

In [43]:
base_url = "https://www.canadiantire.ca/en/promotions/clearance.html?page="

service = Service("C:\Program Files\Common Files\ChromeDriver\chromedriver.exe")
driver = webdriver.Chrome(service=service)

driver.get(base_url)
time.sleep(5)
html_content = driver.page_source
html_soup = soup(html_content, 'html.parser')

total_results_element = html_soup.find('span', class_='nl-filters__results')
total_results_text = total_results_element.text if total_results_element else '0'

total_results = int(re.search(r'\d+', total_results_text).group())

items_per_page = 24
total_pages = math.ceil(total_results / items_per_page)

product_data = []

for page_number in range(1, total_pages + 1):
    url = base_url + str(page_number)
    driver.get(url)
    time.sleep(5)

    try:
        not_right_now_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "kplDeclineButton"))
        )
        not_right_now_button.click()
    except:
        pass

    html_content = driver.page_source
    html_soup = soup(html_content, 'html.parser')

    product_info = html_soup.find_all(class_='nl-product-card__primary-details')

    for product in product_info:
        product_name_element = product.find('div', class_='nl-product-card__title')
        original_price_element = product.find('s', attrs={'aria-hidden': 'true'})
        clearance_price_element = product.find('span', class_='nl-price--total--red')
        rating_element = product.find('div', class_='bv_text')
        image_element = product.find('div', class_='nl-product-card__image-wrap')

        product_name = product_name_element.text.strip() if product_name_element else 'N/A'
        original_price = original_price_element.text.strip() if original_price_element else 'N/A'
        clearance_price = clearance_price_element.text.strip() if clearance_price_element else 'N/A'
        rating = rating_element.text.strip() if rating_element else 'N/A'
        product_code_element = product.find('p', class_='nl-product__code', attrs={'aria-hidden': 'true'})
        product_code = product_code_element.get_text(strip=True).lstrip('#') if product_code_element else 'N/A'

        product_data.append({
            'Product Name': product_name,
            'Original Price': original_price,
            'Clearance Price': clearance_price,
            'Rating': rating,
            'Product Code': product_code
        })

driver.quit()

print(f"Total number of results: {total_results}")
print(f"Total number of pages: {total_pages}")

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=115.0.5790.102)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0111A813+48355]
	(No symbol) [0x010AC4B1]
	(No symbol) [0x00FB5358]
	(No symbol) [0x00F9D293]
	(No symbol) [0x00FFE37B]
	(No symbol) [0x0100C473]
	(No symbol) [0x00FFA536]
	(No symbol) [0x00FD82DC]
	(No symbol) [0x00FD93DD]
	GetHandleVerifier [0x0137AABD+2539405]
	GetHandleVerifier [0x013BA78F+2800735]
	GetHandleVerifier [0x013B456C+2775612]
	GetHandleVerifier [0x011A51E0+616112]
	(No symbol) [0x010B5F8C]
	(No symbol) [0x010B2328]
	(No symbol) [0x010B240B]
	(No symbol) [0x010A4FF7]
	BaseThreadInitThunk [0x76B77D59+25]
	RtlInitializeExceptionChain [0x7754B79B+107]
	RtlClearBits [0x7754B71F+191]


In [ ]:
df = pd.DataFrame(product_data)
df.info()

In [ ]:
df.head()